<a href="https://colab.research.google.com/github/C01day/Colab/blob/main/Multilayer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 准备数据
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = tf.keras.datasets.mnist.load_data()

# 拆开
(train_images, train_labels), (test_images, test_labels) = mnist

# 铺平
A,B,C = train_images.shape
train_images = train_images.reshape((A,B*C))

A,B,C = test_images.shape
test_images = test_images.reshape((A,B*C))

train_images = train_images/256
test_images = test_images/256

# one-hot，例如[3]->[0,0,0,1,0,0,0,0,0,0]
train_labels_onehot = tf.keras.utils.to_categorical(train_labels)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels)

# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 100
display_step = 1

Instructions for updating:
non-resource variables are not supported in the long term
11501568/11490434 [==============================] - 0s 0us/step


# tensorflow

In [ ]:
# tensorflow
# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
# 占位符，先分配内存空间，之后每批数据都在此空间内训练
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
# 输入层、隐藏层1、隐藏层2、输出层
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
# 全连接，每一层的bias参数个数等于下一层的神经元数
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
# x是输入层数据
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) # wx+b
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
# 输出层作为model
logits = multilayer_perceptron(X)

# Define loss and optimizer
# softmax_cross_entropy_with_logits计算预测值和标签的交叉熵，返回数组，reduce_mean再求平均值
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    # 将所有数据分为多个batch训练，记为一轮（相当于Mini-Batch梯度下降）
    # 训练epoch轮次
    num_examples = len(train_images)
    for epoch in range(training_epochs):
        avg_cost = 0.
        # total_batch = int(mnist.train.num_examples/batch_size)
        total_batch = int(num_examples/batch_size)

        # Loop over all batches
        for i in range(total_batch):

            # batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = train_images[i*batch_size : min((i+1)*batch_size , num_examples)]
            batch_y = train_labels_onehot[i*batch_size : min((i+1)*batch_size , num_examples)]

            # Run optimization op (backprop) and cost op (to get loss value)
            # feed_dict将每批数据传给占位符
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})

            # Compute average loss
            avg_cost += c / total_batch

        # Display logs per epoch step
        # 每个epoch输出一次损失
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    # 将输出用softmax转为0-1之间的概率
    pred = tf.nn.softmax(logits)  # Apply softmax to logits # n*10

    # argmax得到数组中最大数的下标（即最大概率的种类）
    # equal比较是否相等，返回0或1
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1)) # n*1

    # Calculate accuracy
    # cast转换数据类型
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: test_images, Y: test_labels_onehot}))  

# sklearn

In [ ]:
# sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
cls = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
print('准确率： %s' % cross_val_score(cls, train_images, train_labels, cv=5).mean())

# keras

In [ ]:
# keras
import tensorflow as tf

# 注意传入的数据需为float型
# Dense layer can only be built with a floating-point dtype
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=10, # patience：能够容忍多少个epoch内都没有improvement
                                                    mode='min')

model.compile(loss=tf.keras.losses.categorical_crossentropy, #'categorical_crossentropy'损失
                optimizer=tf.optimizers.Adam(), #优化器
                metrics=['accuracy']) #指标 #MeanAbsoluteError、MeanSquaredError... #'accuracy'(depends on the loss function) = tf.keras.metrics.CategoricalAccuracy()
            
model.fit(train_images, train_labels_onehot, epochs=training_epochs, batch_size=batch_size,
                      # validation_data=(test_images, test_labels_onehot),
                      validation_split=0.2,
                      callbacks=[early_stopping])
 
model.evaluate(test_images, test_labels_onehot, verbose=0) # loss value & metrics values

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - ETA: 0s - loss: 0.2677 - acc: 0.9221

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


48000/48000 [==============================] - 2s 42us/sample - loss: 0.2677 - acc: 0.9221 - val_loss: 0.1263 - val_acc: 0.9622
Epoch 2/100
48000/48000 [==============================] - 2s 40us/sample - loss: 0.0986 - acc: 0.9710 - val_loss: 0.0914 - val_acc: 0.9732
Epoch 3/100
48000/48000 [==============================] - 2s 41us/sample - loss: 0.0653 - acc: 0.9803 - val_loss: 0.0847 - val_acc: 0.9747
Epoch 4/100
48000/48000 [==============================] - 2s 41us/sample - loss: 0.0445 - acc: 0.9860 - val_loss: 0.0876 - val_acc: 0.9753
Epoch 5/100
48000/48000 [==============================] - 2s 41us/sample - loss: 0.0335 - acc: 0.9893 - val_loss: 0.0938 - val_acc: 0.9744
Epoch 6/100
48000/48000 [==============================] - 2s 40us/sample - loss: 0.0252 - acc: 0.9919 - val_loss: 0.1058 - val_acc: 0.9718
Epoch 7/100
48000/48000 [==============================] - 2s 41us/sample - loss: 0.0201 - acc: 0.9932 - val_loss: 0.0874 - val_acc: 0.9782
Epoch 8/100
48000/48000 [=======

[0.10329462310847812, 0.9774]